## Climate indices

The current production version of PAVICS provides ICCLIM indices through the FlyingPigeon server, which includes over fourty different indices. In the example below, we're calling on one of these indices on a small test file available on a public URL, but it is also possible to send local files to the server. 

Another implementation of climate indices is in the works. It relies on the xarray+dask backend to parallelize computations in the background. The library of climate indices is [xclim](https://github.com/Ouranosinc/xclim) and the WPS server hosting it is [finch](https://github.com/bird-house/finch), but is not yet in production.

In [1]:
%matplotlib inline
import os
from birdy import WPSClient

verify_ssl = True if 'DISABLE_VERIFY_SSL' not in os.environ else False

url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/flyingpigeon/wps'
fp = WPSClient(url, verify=verify_ssl)

In [2]:
help(fp.icclim_tx)

Help on method icclim_tx in module birdy.client.base:

icclim_tx(resource=None, grouping='yr') method of birdy.client.base.WPSClient instance
    Calculates the TX indice: mean of daily maximum temperature.
    
    Parameters
    ----------
    resource : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-tar`, :mimetype:`application/zip`
        NetCDF Files or archive (tar/zip) containing netCDF files.
    grouping : {'day', 'mon', 'sem', 'yr', 'ONDJFM', 'AMJJAS', 'DJF', 'MAM', 'JJA', 'SON', ...}string
        Temporal group over which the index is computed.
    
    Returns
    -------
    output_netcdf : ComplexData:mimetype:`application/x-netcdf`
        The indicator values computed on the original input grid.
    output_log : ComplexData:mimetype:`text/plain`
        Collected logs during process run.



In [3]:
# NBVAL_RAISES_EXCEPTION
thredds = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/fileServer/birdhouse/testdata/flyingpigeon/'
resp = fp.icclim_tx(resource=thredds+'cmip3/tas.sresb1.giss_model_e_r.run1.atm.da.nc', grouping='yr')
tx, log = resp.get(asobj=True)
tx

ServiceException: Maximum number of parallel running processes reached. Please try later.

Let's now check that we indeed computed an annual maximum temperature. Since the file object is already opened using `netCDF`, we need to tell `xarray` to create a dataset from a `NetCDF4DataStore`. 

In [4]:
# NBVAL_RAISES_EXCEPTION
import xarray as xr
ds = xr.open_dataset(xr.backends.NetCDF4DataStore(tx))
ds.icclim_TX.isel(lat=0, lon=0).plot()

NameError: name 'tx' is not defined